In [ ]:
import sys
from PIL import Image
from torchvision import models, transforms, utils
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import os

import numpy as np

import matplotlib.pyplot as plt

sys.path.insert(0, "/home/kylecshan/Kaushik/retrieval/")
from model import load_model

In [ ]:
model_name = "delf"
model_path = "/home/kylecshan/saved/06_01_2019/triplet/delf/model1_epoch7.pth"

In [ ]:
model = load_model(model_name, model_path)

In [ ]:

image_dir = "/home/kylecshan/data/images224/train_ms2000_v5/test/122284"
i = 0
allimages = []
for root, folder, file in os.walk(image_dir):
    for f in file:
        allimages.append(os.path.join(root, f))
allimages

In [ ]:
vis_transform = transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
            ])

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    

def viewimgtensor(img):
    img = torch.transpose(torch.transpose(img, 0, 2), 0, 1)
    plt.imshow(img)

In [ ]:
convmodel = nn.Sequential(*list(model.module.children())[:2])
attnpool = list(model.module.children())[2]

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def get_attention(img):
    imginput = normalize(img)
    imginput = imginput.view(-1,3,224,224).to(device)
    features = convmodel(imginput)
    
    scores = attnpool.conv2(attnpool.conv1(features).clamp(min=0))
    scores = attnpool.activ(scores)
    
    scores = nn.Upsample(scale_factor=32, mode='nearest')(scores)
    scores = scores.squeeze().view(1,224,224)
    
    scores = scores/torch.max(scores)
    scores = scores.repeat(3,1,1).cpu()
    return(scores)

In [ ]:
#i = 112
#i = 280
#i = 468
#i += 4
i = 2
image_path = allimages[i]
img = Image.open(image_path)
img = vis_transform(img)

attn = get_attention(img)
grid = utils.make_grid([img, attn, img*attn], nrow=3)
plt.imshow(grid.numpy().transpose(1,2,0))

In [ ]:
i

In [ ]:
img.size()